# WikiTableQuestions

### Uploading Dataset to huggingface


In [ ]:
import os
from dotenv import load_dotenv

from huggingface_hub import HfApi

# Load the .env file
load_dotenv()

# Get the token
hf_token = os.getenv("HF_TOKEN")

api = HfApi(token=hf_token)
api.upload_large_folder(
    folder_path="tabmwp/",
    repo_id="TableSenseAI/TabMWP",
    repo_type="dataset",
)


Failed to commit: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/TableSenseAI/TabMWP/commit/main (Request ID: Root=1-68497b30-211003fa0a983e556c9b9a00;8cb4a42d-e34a-46fb-b6ec-38f17010953d)

You have been rate-limited; you can retry this action in 39 minutes. If you're a new user, your limits will raise progressively over time. Get in touch with us at website@huggingface.co if you need access now.
Failed to commit 20 files at once. Will retry with less files in next batch.
Failed to commit: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/TableSenseAI/TabMWP/commit/main (Request ID: Root=1-68497b31-28dc65042c83431466a491e8;e5e92535-265d-4209-b5df-35dad4783fde)

You have been rate-limited; you can retry this action in 39 minutes. If you're a new user, your limits will raise progressively over time. Get in touch with us at website@huggingface.co if you need access now.
Failed to commit 20 files at once. Will retry with less

### Preprocessing Dataset

In [ ]:
import csv
import json
import os

input_tsv = 'WikiTableQuestions-data/data/pristine-unseen-tables.tsv'         # Replace with your input file
output_json = 'examples-test.json'

data = []

with open(input_tsv, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter='\t')
    for idx, row in enumerate(reader):
        base_path, _ = os.path.splitext(row["context"])
        entry = {
            "id": f"nu-{idx}",
            "utterance": row["utterance"],
            "target_value": row["targetValue"],
            "context": {
                "csv": base_path + ".csv",
                "html": base_path + ".html",
                "tsv": base_path + ".tsv"
            }
        }
        data.append(entry)

with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2)

print(f"Wrote {len(data)} entries to {output_json}")


### TabMWP Preprocessing

In [ ]:
import json
import pandas as pd
import os

input_json1 = 'C:/Users/Marco/workspace/TabMWP/PromptPG/data/tabmwp/problems_dev.json'         # Replace with your input file
input_json2 = 'C:/Users/Marco/workspace/TabMWP/PromptPG/data/tabmwp/problems_test.json' 
output_json = 'tabmwp/examples-test.json'

data = []
table_id = 23059
location = 'tabmwp/'

with open(input_json1, 'r') as f1, open(input_json2, 'r') as f2:
    in1 = json.load(f1)
    in2 = json.load(f2)
    orig = {**in1, **in2}
    for key, value in orig.items():
        df = pd.DataFrame(value["table_for_pd"])
        basepath = f"csv/{table_id}"
        os.makedirs(location+'csv', exist_ok=True)
        df.to_csv(location+ basepath + ".csv", index=False)
        df.to_html(location+basepath+".html", index=False)
        df.to_csv(location+basepath+".tsv", sep="\t", index=False)
        
        entry = {
            "id": f"nu-{key}",
            "utterance": value["question"],
            "target_value": value["answer"],
            "context": {
                "csv": basepath+ ".csv",
                "html": basepath+".html",
                "tsv": basepath+".tsv"
            }
        }
        data.append(entry)
        table_id += 1

with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2)

print(f"Wrote {len(data)} entries to {output_json}")